**Import Packages**

In [1]:
from langchain.document_loaders import TextLoader                      # Load the text
from langchain.text_splitter import RecursiveCharacterTextSplitter     # Chunks
from langchain_ollama import OllamaLLM, OllamaEmbeddings               # llm model and embedding model  from "Ollama"
from langchain.vectorstores import Chroma                              # Store the vectors
from langchain.prompts import PromptTemplate                           # Prompt template
from langchain.chains import RetrievalQA
import warnings
warnings.filterwarnings('ignore')

**Step-1 : Load the TextFile dataset**

- Loading Text file Dataset

In [2]:
from langchain.document_loaders import TextLoader                      # Load the text

file_path="State_union.txt"
loader = TextLoader(file_path,encoding="utf-8-sig")
raw_documents = loader.load()
print(len(raw_documents))

1


In [3]:
print(raw_documents[0].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.

Last year COVID-19 kept us apart. This year we are finally together again.

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.

With a duty to one another to the American people to the Constitution.

And with an unwavering resolve that freedom will always triumph over tyranny.

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.

He met the Ukrainian people.

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.

Groups of citizens blocking tanks with their bodies. Everyone from s

**Step-2 : Splitting into Chunks**

- Because language models (LLMs) like LLaMA, Gemini AI, GPT, etc.,

- Have a limit on how much text they can handle at once — this is called the context window.

In [19]:
# Split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter     # Chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(raw_documents)

print(f'No.of Chunks : {len(chunks)}')
print(f'1st Chunk : {chunks[0].page_content}')
print(f'No.of Characters in 1st Chunk : {len(chunks[0].page_content)}')

No.of Chunks : 46
1st Chunk : Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.

Last year COVID-19 kept us apart. This year we are finally together again.

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.

With a duty to one another to the American people to the Constitution.

And with an unwavering resolve that freedom will always triumph over tyranny.

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.

He met the Ukrainian people.

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.
No.of Characters in 1st Chunk : 930


In [20]:
for i in range(len(chunks)):
    print(f'No.of Characters in {i+1}st Chunk : {len(chunks[i].page_content)}')

No.of Characters in 1st Chunk : 930
No.of Characters in 2st Chunk : 975
No.of Characters in 3st Chunk : 958
No.of Characters in 4st Chunk : 991
No.of Characters in 5st Chunk : 934
No.of Characters in 6st Chunk : 910
No.of Characters in 7st Chunk : 945
No.of Characters in 8st Chunk : 997
No.of Characters in 9st Chunk : 990
No.of Characters in 10st Chunk : 950
No.of Characters in 11st Chunk : 898
No.of Characters in 12st Chunk : 961
No.of Characters in 13st Chunk : 924
No.of Characters in 14st Chunk : 921
No.of Characters in 15st Chunk : 981
No.of Characters in 16st Chunk : 925
No.of Characters in 17st Chunk : 942
No.of Characters in 18st Chunk : 912
No.of Characters in 19st Chunk : 974
No.of Characters in 20st Chunk : 679
No.of Characters in 21st Chunk : 994
No.of Characters in 22st Chunk : 877
No.of Characters in 23st Chunk : 956
No.of Characters in 24st Chunk : 917
No.of Characters in 25st Chunk : 904
No.of Characters in 26st Chunk : 941
No.of Characters in 27st Chunk : 940
No.of Char

**Step-3 : Converting Chunks into Embeddings**

- Because embeddings let us search by meaning, not just exact words.

In [21]:
# Set up ollama embedding model
from langchain_ollama import OllamaEmbeddings               # embedding model  from "Ollama"

embedding = OllamaEmbeddings(model="llama3.2:1b")

**Step-4 : Storing Embeddings in a vectorDB**

- Embeddings should be stored in a VectorDB

- We have different types of vectorDB

- Here we are using ChromaDB to store embeddings

In [22]:
from langchain.vectorstores import Chroma                              #  Using chromaDB to store embeddings

vectordb = Chroma.from_documents(chunks,embedding)

# Use the vectorstore as a retriever
# Make a retriever from my vector database, and when I ask a question, return the top 5 most relevant chunks.
retriever = vectordb.as_retriever()    

**Step-5 : Loading LLM model**

In [23]:
from langchain_ollama import OllamaLLM              # llm model from "Ollama"

llm=OllamaLLM(model="llama3.2:1b",base_url="http://localhost:11434")      # Model name "llama3.2:1b"
llm

OllamaLLM(model='llama3.2:1b', base_url='http://localhost:11434')

**Step-6 : Prompting**

- By using prompt we can control model

- Prompts Help You Get the Desired Output

In [24]:
from langchain.prompts import PromptTemplate                           # Prompt template

prompt = PromptTemplate.from_template("""
    You are a knowledgeable and concise assistant.
    
    Your task is to answer the question using only the information provided in the context below.
    
    --- Context ---
    {context}
    ----------------
    
    Question: {question}
    
    Answer:
""")

print(prompt.template)


    You are a knowledgeable and concise assistant.
    
    Your task is to answer the question using only the information provided in the context below.
    
    --- Context ---
    {context}
    ----------------
    
    Question: {question}
    
    Answer:



**Step-7 : Building Retrival Chain**

- User query → Retriever finds relevant chunks → chain_type merges them → LLM answers

In [25]:
# Build the RetrievalQA chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
            llm=llm,                              # 👈 the LLM (like GPT-3.5 or LLaMA) that answers the question
            retriever=retriever,                  # 👈 the thing that searches and gives relevant chunks from your document
            chain_type="stuff",                   # 👈 method to combine chunks into one input for the LLM
            chain_type_kwargs={"prompt": prompt}  # 👈 your custom prompt format
)

**Step-8 : Asking Question**

- User question → Embedding → Find top-k similar chunks from vector DB → Send to LLM → Generate answer




- User question
-    ↓
- Convert to embedding (vector)              🔹 Done by: Embedding model (e.g., OllamaEmbeddings)
-    ↓
- Find top-k similar chunks from vector DB   🔹 Done by: as_retriever (e.g., Chroma retriever)
-    ↓
- Send retrieved chunks + question to LLM    🔹 Done by: RetrievalQA chain (or manually via prompt)
-    ↓
- Generate final answer                      🔹 Done by: LLM (e.g., llama3.2:1b)



In [28]:
# Ask a question
query = input("Enter Your Question : ")
response = qa_chain.run(query)

print("Answer:", response)

# What did the president say about Ketanji Brown Jackson?
# What support is being provided to Ukraine?
# What did the President say about COVID-19?
# What is the American Rescue Plan and how did it help people?

Enter Your Question :  What did the President say about COVID-19?


Answer: The President mentioned that "we have left no one behind or ignored anyone's needs as we move forward." They also stated that they can end the shutdown of schools and businesses, and will do everything within their power to be ready if a new variant is detected. Additionally, they emphasized the availability of free tests, including those for COVID-19, vaccines, masks, and pills, which were ordered through "covidtests.gov".


# All Together

In [52]:
from langchain.document_loaders import TextLoader                      # Load the text
from langchain.text_splitter import RecursiveCharacterTextSplitter     # Chunks
from langchain_ollama import OllamaLLM, OllamaEmbeddings               # llm model and embedding model  from "Ollama"
from langchain.vectorstores import Chroma                              # Store the vectors
from langchain.prompts import PromptTemplate                           # Prompt template
from langchain.chains import RetrievalQA
import warnings
warnings.filterwarnings('ignore')

######################################### Text File Loader ################################################33
file_path="State_union.txt"
loader = TextLoader(file_path,encoding="utf-8-sig")
raw_documents = loader.load()

######################################### Splitting into Chunks ################################################33
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=150)
chunks = text_splitter.split_documents(raw_documents)

######################################### Loading Embedding Model ################################################33
embedding = OllamaEmbeddings(model="llama3.2:1b")

######################################### Storing Embeddings into vectorDB ################################################33
vectordb = Chroma.from_documents(chunks,embedding)
retriever = vectordb.as_retriever()  

######################################### Loading LLM Model ################################################33
llm=OllamaLLM(model="llama3.2:1b",base_url="http://localhost:11434") 

######################################### Prompting ################################################33
prompt = PromptTemplate.from_template("""
    You are a knowledgeable and concise assistant.
    
    Your task is to answer the question using only the information provided in the context below.
    
    Do not use any external knowledge, and do not make assumptions.
    
    If the answer is not explicitly stated in the context, reply with:
    "I don't know based on the provided context."
    
    --- Context ---
    {context}
    ----------------
    
    Question: {question}
    
    Answer:
""")

######################################### Settingup QA Chain ################################################33
qa_chain = RetrievalQA.from_chain_type(
            llm=llm,                              # 👈 the LLM (like GPT-3.5 or LLaMA) that answers the question
            retriever=retriever,                  # 👈 the thing that searches and gives relevant chunks from your document
            chain_type="stuff",                   # 👈 method to combine chunks into one input for the LLM
            chain_type_kwargs={"prompt": prompt}  # 👈 your custom prompt format
)

######################################### Asking Question ################################################33
query = input("Enter Your Question : ")
response = qa_chain.run(query)

print("Answer:", response)

# What did the president say about economy?
# What steps did the President propose to fight inflation?
# What did the President say about creating jobs and supporting American manufacturing?

Enter Your Question :  What did the president say about economy?


Answer: The president mentioned that their "economy grew at a rate of 5.7% last year, the strongest growth in nearly 40 years... bringing fundamental change to an economy that hasn’t worked for the working people of this nation for too long."
